In [ ]:
#@title  **Comprehensive Community Analysis and Robustness Check**


# --- [الخطوة 0: تثبيت المكتبات] ---
print("--- [Step 0: Installing Libraries] ---")
!pip install -qq pandas networkx python-igraph leidenalg python-louvain scikit-learn
print("Libraries installed successfully.\n")
!git clone https://github.com/NoorBayan/Amthal.git
%cd Amthal
from utility import *
from IPython.display import display, HTML, clear_output


# --- [الخطوة 1: استيراد المكتبات] ---
print("--- [Step 1: Importing Libraries] ---")
import pandas as pd
import networkx as nx
import igraph as ig
import leidenalg as la
import community.community_louvain as community_louvain
from sklearn.metrics.cluster import normalized_mutual_info_score
from collections import Counter
from itertools import combinations
import ast
print("Libraries imported successfully.\n")


# --- [الخطوة 2: تحميل وتجهيز البيانات] ---
print("--- [Step 2: Loading Data] ---")
INSTANCES_FILE_PATH = '/content/Amthal/data/processed/instances.csv'
try:
    df = pd.read_csv(INSTANCES_FILE_PATH, sep='\t', encoding='utf-16')
    df['Dominant_Concept']=[Dominant_Concept_dic[i]['en'] for i in df.Dominant_Concept]
    df['Valence']=[Valence_dic[i]['ar'] for i in df.Valence]
    df['Revelation_Phase']=[Revelation_Phase_dic[i]['ar'] for i in df.Revelation_Phase]
    df['Abstraction_Level']=[Abstraction_Level_dic[i]['ar'] for i in df.Abstraction_Level]

    print(f"Data loaded successfully. Found {len(df)} instances.\n")
except FileNotFoundError:
    print(f"⚠️ ERROR: File not found at '{INSTANCES_FILE_PATH}'. Please check the path and try again.")
    exit()


# --- [الخطوة 3: بناء الشبكة الدقيقة] ---
print("--- [Step 3: Building the Accurate Network] ---")
G_nx = nx.Graph()
verses = df.groupby(['Sura_No', 'Aya_No'])

for name, group in verses:
    concepts_in_verse = list(set(group['Dominant_Concept']))
    if len(concepts_in_verse) > 1:
        verse_text = group['vers_text'].iloc[0]
        verse_length = len(str(verse_text).split())
        weight = 1 / verse_length if verse_length > 0 else 0

        for concept1, concept2 in combinations(concepts_in_verse, 2):
            if G_nx.has_edge(concept1, concept2):
                G_nx[concept1][concept2]['weight'] += weight
            else:
                G_nx.add_edge(concept1, concept2, weight=weight)
print(f"Network built. It has {G_nx.number_of_nodes()} nodes and {G_nx.number_of_edges()} edges.\n")


# --- [الخطوة 4: تحليل استقرار Leiden] ---
print("--- [Step 4: Leiden Stability Analysis (100 runs)] ---")
G_ig = ig.Graph.from_networkx(G_nx)
num_runs = 100
all_partitions_leiden = []
for i in range(num_runs):
    partition = la.find_partition(G_ig, la.RBConfigurationVertexPartition, weights='weight', seed=i)
    all_partitions_leiden.append(partition)

partition_strings = []
for p in all_partitions_leiden:
    # ✅  التصحيح: استخدام '_nx_name' بدلاً من 'name'
    sorted_partition = sorted([sorted(G_ig.vs[community]['_nx_name']) for community in p])
    partition_strings.append(str(sorted_partition))

partition_counts = Counter(partition_strings)
most_common_partition_str, count = partition_counts.most_common(1)[0]
stability_percentage = (count / num_runs) * 100
stable_partition_leiden_obj = all_partitions_leiden[partition_strings.index(most_common_partition_str)]

print("Leiden Stability Results:")
print(f"The most stable partition was found {count} out of {num_runs} times ({stability_percentage:.2f}% stability).")
print(f"Number of communities in stable partition: {len(stable_partition_leiden_obj)}")
print(f"Modularity Score (Q) of stable partition: {stable_partition_leiden_obj.modularity:.4f}\n")


# --- [الخطوة 5: فحص القوة بالمقارنة مع Louvain] ---
print("--- [Step 5: Robustness Check vs. Louvain] ---")
partition_louvain_dict = community_louvain.best_partition(G_nx, weight='weight', random_state=42)

node_order = list(G_nx.nodes())
louvain_membership = [partition_louvain_dict[node] for node in node_order]

stable_leiden_membership = [0] * len(node_order)
for i, community in enumerate(stable_partition_leiden_obj):
    for vertex_index in community:
        node_name = G_ig.vs[vertex_index]['_nx_name'] # ✅ التأكد من استخدام '_nx_name' هنا أيضاً
        node_idx_in_order = node_order.index(node_name)
        stable_leiden_membership[node_idx_in_order] = i

nmi_score = normalized_mutual_info_score(stable_leiden_membership, louvain_membership)
print("Comparison Results:")
print(f"Leiden found {len(set(stable_leiden_membership))} communities.")
print(f"Louvain found {len(set(louvain_membership))} communities.")
print(f"✅ Normalized Mutual Information (NMI) Score: {nmi_score:.4f}\n")


# --- [الخطوة 6: طباعة التقرير النهائي الشامل] ---
print("="*20 + " FINAL REPORT " + "="*20)
print("This report contains all the numbers needed for the paper and supplementary material.\n")
print("--- MAIN RESULT (from Leiden) ---")
print(f"Number of Communities: {len(stable_partition_leiden_obj)}")
print(f"Modularity (Q): {stable_partition_leiden_obj.modularity:.4f}")
print(f"Stability: {stability_percentage:.2f}% ({count}/{num_runs} runs)")

print("\n--- Community Details (Leiden) ---")
stable_partition_list = ast.literal_eval(most_common_partition_str)
for i, community_nodes in enumerate(stable_partition_list):
    print(f"  Community {i+1}: {', '.join(community_nodes)}")

print("\n\n--- ROBUSTNESS CHECK RESULT (vs. Louvain) ---")
print("This part is for the supplementary material.\n")
print(f"Number of communities found by Louvain: {len(set(louvain_membership))}")
print(f"✅ NMI between Leiden and Louvain: {nmi_score:.4f}")

print("\n--- Louvain Partition Details (for comparison) ---")
louvain_communities = {}
for node, comm_id in partition_louvain_dict.items():
    if comm_id not in louvain_communities:
        louvain_communities[comm_id] = []
    louvain_communities[comm_id].append(node)

for comm_id, nodes in sorted(louvain_communities.items()):
    print(f"  Community {comm_id}: {sorted(nodes)}")

print("="*54)

--- [Step 0: Installing Libraries] ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 95.6 MB/s eta 0:00:00
Libraries installed successfully.

Cloning into 'Amthal'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 101 (delta 14), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 2.84 MiB | 5.10 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/Amthal
--- [Step 1: Importing Libraries] ---
Libraries imported successfully.

--- [Step 2: Loading Data] ---
--- [Step 1: Loading Data] ---
Data loaded successfully. Found 4078 instances.

--- [Step 3: Building the Accurate Network] ---
Network built. It has 21 nodes and 150 edges.

--- [Step 4: Leiden Stability Analysis (100 runs)] ---
Leiden Stability Results:
The most stable partition was found 91 out of 100 times (91.